In [1]:
import pandas as pd

In [2]:
#schema = ['uid','device','song_id','song_type','song_name','singer','play_time','song_length','paid_flag','date']
df = pd.read_csv('../data/play_ds.csv',delimiter=',')

/home/zhengyzhou040820/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head(5)

,uid,device,song_id,date,play_time,song_length
0,168550892,ar,23491655,2017-03-30,254,254.0
1,168549788,ip,295469,2017-03-30,16,242.0
2,168532580,ar,21762903,2017-03-30,67,137.0
3,168543026,ar,6623026,2017-03-30,0,0.0
4,168532580,ar,21762903,2017-03-30,67,137.0


In [4]:
def calculateTime(row):
    if int(row['song_length']) == 0:
        #print("processed")
        return 0
    else:
        #print("processed")
        return float(row['play_time_2']) / float(row['song_length'])

In [5]:
# fill na with 0
df['play_time'].fillna('0', inplace=True)

In [6]:
def cleanPlayTime(row):
    if row['play_time'] == 0:
        return 0
    if '>' in row['play_time'] :
        # print s.split('>')
        return row['play_time'].split('>')[0]
    else:
        return row['play_time']

In [7]:
df['play_time_2'] = df.apply(cleanPlayTime,axis = 1)

In [8]:
df.head(5)

,uid,device,song_id,date,play_time,song_length,play_time_2
0,168550892,ar,23491655,2017-03-30,254,254.0,254
1,168549788,ip,295469,2017-03-30,16,242.0,16
2,168532580,ar,21762903,2017-03-30,67,137.0,67
3,168543026,ar,6623026,2017-03-30,0,0.0,0
4,168532580,ar,21762903,2017-03-30,67,137.0,67


In [9]:
df[['play_time_2']] = df[['play_time_2']].astype(float)

In [10]:
df['play_time_2'].fillna('0', inplace=True)

In [11]:
df['song_length'].fillna('0', inplace=True)

In [12]:
df.iloc[[34027]]

,uid,device,song_id,date,play_time,song_length,play_time_2
34027,168682366,ar,,2017-03-30,0,0,0.0


In [13]:
df['play_ratio'] = df.apply(calculateTime,axis = 1)

In [14]:
df.head(5)

,uid,device,song_id,date,play_time,song_length,play_time_2,play_ratio
0,168550892,ar,23491655,2017-03-30,254,254,254.0,1.000000
1,168549788,ip,295469,2017-03-30,16,242,16.0,0.066116
2,168532580,ar,21762903,2017-03-30,67,137,67.0,0.489051
3,168543026,ar,6623026,2017-03-30,0,0,0.0,0.000000
4,168532580,ar,21762903,2017-03-30,67,137,67.0,0.489051


In [16]:
df.to_csv('../data/play_ds_extention.csv')

In [17]:
df_rate = df.groupby(['uid','song_id'])

In [18]:
df_rate = df.drop(['device','date','play_time','song_length','play_time_2'], axis=1)

In [19]:
df_rate = df_rate.groupby(['uid', 'song_id']).agg({'play_ratio':'sum'})

In [20]:
df_rate.head(5)

play_ratio
uid   song_id              
12333 703693.0     0.821429
      708428.0     9.682796
      875447.0     8.418803
      5114569.0    7.802120
      5237384.0    8.000000

In [21]:
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
df_rate = df_rate.reset_index()

In [24]:
df_rate.to_csv('../data/play_rate.csv')

In [25]:
# convert uid to a smaller list
user_id_lst = df_rate['uid'].unique() 
user_id_dict = {k: v for v, k in enumerate(user_id_lst)}

# convert song id to a smaller list
song_id_lst = df_rate['song_id'].unique() 
song_id_dict = {k: v for v, k in enumerate(song_id_lst)}

In [26]:
df_rate['user_id_map'] = df_rate.apply(lambda x: user_id_dict[x['uid']],axis = 1)
df_rate['song_id_map'] = df_rate.apply(lambda x: song_id_dict[x['song_id']],axis = 1) 

In [27]:
highest_user_id = df_rate.user_id_map.max() + 1
highest_song_id = df_rate.song_id_map.max() + 1
ratings_mat = sparse.lil_matrix((highest_user_id, highest_song_id)) 
ratings_mat

<53318x346055 sparse matrix of type '<type 'numpy.float64'>'
	with 0 stored elements in LInked List format>

In [28]:
for _, row in df_rate.iterrows():
# subtract 1 from id's due to match 0 indexing 
    ratings_mat[row.user_id_map-1, row.song_id_map-1] = row.play_ratio

In [29]:
del df

In [30]:
# item_item recommendation
utility_mat = ratings_mat
# Item-Item Similarity Matrix
item_sim_mat = cosine_similarity(utility_mat.T)

MemoryError: 

In [ ]:
least_to_most_sim_indexes = np.argsort(item_sim_mat, axis=1)
# Neighborhoods
neighborhood_size = 75
neighborhoods = least_to_most_sim_indexes[:, -neighborhood_size:]

In [ ]:
neighborhoods.shape

In [ ]:
user_id = 200

n_users = utility_mat.shape[0] 
n_items = utility_mat.shape[1]

start_time = time()
items_rated_by_this_user = ratings_mat[user_id].nonzero()[1] 
# Just initializing so we have somewhere to put rating preds 
out = np.zeros(n_items)
for item_to_rate in range(n_items):
    relevant_items = np.intersect1d(neighborhoods[item_to_rate], 
                                    items_rated_by_this_user,
                                    assume_unique=True) 
# assume_unique speeds up intersection 
out[item_to_rate] = ratings_mat[user_id, relevant_items] * \
        item_sim_mat[item_to_rate, relevant_items] / \ 
        item_sim_mat[item_to_rate, relevant_items].sum()

pred_ratings = np.nan_to_num(out)
print(pred_ratings)
print("Execution time: %f seconds" % (time()-start_time))